In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sachi\\Code\\Deep Learning project\\kidney-Disease-Classification-MLflow-DVC\\kidney-Disease-Classification-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sachi\\Code\\Deep Learning project\\kidney-Disease-Classification-MLflow-DVC\\kidney-Disease-Classification-MLflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:

from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
# Define the input shape
input_shape = (224, 224, 3)  # Example input shape for an image with height 224, width 224, and 3 channels (RGB)

# Define the input layer
input_layer = tf.keras.Input(shape=input_shape)

# Define the rest of your model layers
# For example:
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(input_layer)


In [10]:
# Define the input layer
input_layer = tf.keras.Input(shape=(input_shape))

# Define the rest of your model layers
# For example:
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(input_layer)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

# Now, define the output layer using the correct output from the previous layer
num_classes = 2  # Example number of classes
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)



In [11]:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.steps_per_epoch = None  # Initialize steps_per_epoch attribute
        self.validation_steps = None

    def get_base_model(self):
        # Load the base model
        self.base_model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Adjust the output layer for your classification task
        num_classes = 2  # Example number of classes
        output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(self.base_model.output)
        self.model = tf.keras.Model(inputs=self.base_model.input, outputs=output_layer)

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        # Compile the model
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps
        )

        # Save the trained model
        self.save_model(path=self.config.trained_model_path, model=self.model)

In [12]:

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-04-06 08:56:24,306: INFO: yaml file: config\config.yaml loaded successfully]
[2024-04-06 08:56:24,315: INFO: yaml file: params.yaml loaded successfully]
[2024-04-06 08:56:24,319: INFO: created directory at: artifacts]
[2024-04-06 08:56:24,320: INFO: created directory at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/100
24/24 [==============================] - 94s 4s/step - loss: 0.5568 - accuracy: 0.7823 - val_loss: 0.4982 - val_accuracy: 0.8065
Epoch 2/100
24/24 [==============================] - 96s 4s/step - loss: 0.4949 - accuracy: 0.8441 - val_loss: 0.4065 - val_accuracy: 0.9570
Epoch 3/100
24/24 [==============================] - 89s 4s/step - loss: 0.4785 - accuracy: 0.8602 - val_loss: 0.4816 - val_accuracy: 0.8065
Epoch 4/100
24/24 [==============================] - 94s 4s/step - loss: 0.4703 - accuracy: 0.8575 - val_loss: 0.4600 - val_accuracy: 0.8387
Epoch 5/100
24/24 [==============================] - 97s 

KeyboardInterrupt: 